In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

This code will take in three files that will be different sheets in the market characterization workbook. It will then multiply these values together to get to the initial count of every piece of equipment(condition) in the potential study.

In [6]:
#future cleaning functions here

In [41]:
#@task
def read_in_market_characterization_sheets(file_path: str = './input/02_market_characterization.xlsx') -> tuple:
    """Read market characterization sheets from the Excel file and return individual dataframes"""
    # Define the sheets we want to read
    sheets = ['customer_building_type_count', 'equipment_count_per_building', 'efficiency_level_breakout']
    
    # Read all sheets at once
    all_dfs = pd.read_excel(file_path, sheet_name=sheets, header=0)
    
    # Extract individual dataframes
    customer_building_type_count = all_dfs[sheets[0]]
    equipment_count_per_building = all_dfs[sheets[1]]
    efficiency_level_breakout = all_dfs[sheets[2]]
    
    # Required columns to check
    required_columns = {"single_family", "multi_family"}
    
    # Check each dataframe for required columns
    for sheet_name, df in zip(sheets, [customer_building_type_count, equipment_count_per_building, efficiency_level_breakout]):
        if not required_columns.issubset(df.columns):
            raise ValueError(
                f"Sheet '{sheet_name}' must contain {', '.join(required_columns)} columns"
            )
    
    return customer_building_type_count, equipment_count_per_building, efficiency_level_breakout

In [47]:
customer_building_type_count, equipment_count_per_building, efficiency_level_breakout = read_in_market_characterization_sheets()

In [43]:
#Parameters
building_types = ["single_family", "multi_family", "single_family_li", "multi_family_li"]

Now lets combine all the Files to get what we want to append to the conditions database

In [48]:
# Create a new dataframe with the same structure as equipment_count_per_building
equipment_count_total = equipment_count_per_building.copy()

# Multiply each building type column
for building_type in building_types:
    # Multiply the equipment counts by the corresponding customer count
    equipment_count_total[building_type] = (equipment_count_per_building[building_type] * 
                                          customer_building_type_count[building_type].iloc[0])

equipment_count_total

,competition_group,subgroup,single_family,multi_family,single_family_li,multi_family_li
0,heating_cooling,oil_furnace,20000.0,4000.0,10000.0,2000.0
1,heating_cooling,gas_furnace,20000.0,4000.0,10000.0,2000.0
2,heating_cooling,room_ac,30000.0,6000.0,15000.0,3000.0
3,heating_cooling,central_ac,70000.0,14000.0,25000.0,5000.0
4,heating_cooling,cchp,20000.0,4000.0,10000.0,2000.0
5,refrigeration,full_size,100000.0,24000.0,50000.0,12000.0
6,whole_home,none,100000.0,20000.0,50000.0,10000.0


In [49]:
efficiency_level_breakout

,condition_name,competition_group,subgroup,single_family,multi_family,single_family_li,multi_family_li
0,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,0.20,0.10,0.20,0.10
1,furnace_natural_gas_baseline_residential,heating_cooling,gas_furnace,0.50,0.60,0.50,0.60
2,furnace_natural_gas_efficient_residential,heating_cooling,gas_furnace,0.50,0.40,0.40,0.40
3,room_ac_electricity_baseline_residential,heating_cooling,room_ac,0.70,0.70,0.60,0.60
4,room_ac_electricity_efficient_residential,heating_cooling,room_ac,0.30,0.10,0.10,0.10
5,air_conditioner_electricity_baseline_residential,heating_cooling,central_ac,0.70,1.20,1.00,1.20
6,air_conditioner_electricity_efficient_residential,heating_cooling,central_ac,0.30,0.30,0.30,0.30
7,cchp_electricity_efficient_residential,heating_cooling,cchp,1.00,1.00,1.00,1.00
8,refrigerator_electricity_existing_residential,refrigeration,full_size,0.25,0.25,0.25,0.25
9,refrigerator_electricity_baseline_residential,refrigeration,full_size,0.25,0.25,0.25,0.25


In [ ]:
# Merge equipment_count_total and efficiency_level_breakout on 'condition_name' and 'competition_group'
merged_df = pd.merge(
    efficiency_level_breakout,
    equipment_count_total,
    on=["competition_group", "subgroup"],
    suffixes=("_eff", "_equip")
)

# Multiply building type columns where matches occur
for building_type in building_types:
    merged_df[building_type + "_year_one"] = merged_df[building_type + "_eff"] * merged_df[building_type + "_equip"]

 #Drop columns ending with _eff and _equip from merged_df",
cols_to_drop = [col for col in merged_df.columns if col.endswith('_eff') or col.endswith('_equip')]
df_yr1 = merged_df.drop(columns=cols_to_drop)
df_yr1


,condition_name,competition_group,subgroup,single_family_year_one,multi_family_year_one,single_family_li_year_one,multi_family_li_year_one
0,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,4000.0,400.0,2000.0,200.0
1,furnace_natural_gas_baseline_residential,heating_cooling,gas_furnace,10000.0,2400.0,5000.0,1200.0
2,furnace_natural_gas_efficient_residential,heating_cooling,gas_furnace,10000.0,1600.0,4000.0,800.0
3,room_ac_electricity_baseline_residential,heating_cooling,room_ac,21000.0,4200.0,9000.0,1800.0
4,room_ac_electricity_efficient_residential,heating_cooling,room_ac,9000.0,600.0,1500.0,300.0
5,air_conditioner_electricity_baseline_residential,heating_cooling,central_ac,49000.0,16800.0,25000.0,6000.0
6,air_conditioner_electricity_efficient_residential,heating_cooling,central_ac,21000.0,4200.0,7500.0,1500.0
7,cchp_electricity_efficient_residential,heating_cooling,cchp,20000.0,4000.0,10000.0,2000.0
8,refrigerator_electricity_existing_residential,refrigeration,full_size,25000.0,6000.0,12500.0,3000.0
9,refrigerator_electricity_baseline_residential,refrigeration,full_size,25000.0,6000.0,12500.0,3000.0
